# Генерация исключений

Исключения в Python:
* Доступ к объекту исключения
* Генерация исключений, инструкция raise
* Исключения типа AssertionError
* Вопросы производительности
* Работа с пользовательскими исключениями

## Доступ к объекту исключения

In [1]:
try:
    with open('/file/not/found') as f:
        content = f.read()
except OSError as e:
    print(e.errno, e.strerror) # У каждого типа исключения могут быть свои атрибуты

2 No such file or directory


## Доступ к объекту исключения, атрибут `args`

In [4]:
import os.path

filename = '/file/not/found'
try:
    if not os.path.exists(filename):
        raise ValueError('Файл не существует', filename)
except ValueError as e:
    message, filename = e.args[0], e.args[1]
    print(message, filename)

Файл не существует /file/not/found


## Доступ к стеку вызовов

In [5]:
import traceback

try:
    with open('/file/not/found') as f:
        content = f.read()
except OSError as e:
    traceback.print_exc() # Печать стека вызовов

Traceback (most recent call last):
  File "<ipython-input-5-2a01c49a37e6>", line 4, in <module>
    with open('/file/not/found') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/file/not/found'


## Генерация исключения, инструкция `raise`

In [7]:
try:
    raw = input('Введите некорректное число:')
    if not raw.isdigit():
        raise ValueError('Плохое число', raw)
except ValueError as e:
    print('Некорректное значение!', e)

Введите некорректное число:w20
Некорректное значение! ('Плохое число', 'w20')


## Проброс исключения выше по стеку

In [8]:
try:
    raw = input('Введите некорректное число:')
    if not raw.isdigit():
        raise ValueError('Плохое число', raw)
except ValueError as e:
    print('Некорректное значение!', e)
    # Делегирование обработки исключений выше по стеку. Вызов raise без параметров
    raise

Введите некорректное число:w20
Некорректное значение! ('Плохое число', 'w20')


ValueError: ('Плохое число', 'w20')

## Исключение через `raise from Exception`

Результат: сначала идет `ValueError`, а затем `TypeError`:

In [10]:
try:
    raw = input('Введите некорректное число:')
    if not raw.isdigit():
        raise ValueError('Плохое число', raw)
except ValueError as err:
    raise TypeError('Ошибка xxx') from err

Введите некорректное число:w20


TypeError: Ошибка xxx

## Инструкция `assert`

In [11]:
assert True   # Если что-то выполняется, то исключение не выбрасывается

try:
    # Если что-то возвращает ложь, то будет сгенерировано исключение AssertionError
    assert 1 == 0, '1 не равен 0'
except AssertionError as e:
    traceback.print_exc() # Печать стека вызовов

Traceback (most recent call last):
  File "<ipython-input-11-1e1fad63c562>", line 5, in <module>
    assert 1 == 0, '1 не равен 0'
AssertionError: 1 не равен 0


Инструкция `assert` с флагом `-O`:

In [12]:
def get_user_by_id(id):
    # Если isinstance возвращает False, то генерируется исключение AssertionError:
    assert isinstance(id, int), 'id должен быть целым числом'
    print('Выполняем поиск...')

# Инструкция assert аналогична следующему коду:
def get_user_by_id2(id):
    if not isinstance(id, int):
        raise AssertionError('id должен быть целым числом')
    print('Выполняем поиск...')


if __name__ == '__main__':
    try:
        get_user_by_id('foo') # Вызовет исключение AssertionError
    except AssertionError:
        traceback.print_exc() # Печать стека вызовов

Traceback (most recent call last):
  File "<ipython-input-12-34cdd117eaee>", line 15, in <module>
    get_user_by_id('foo') # Вызовет исключение AssertionError
  File "<ipython-input-12-34cdd117eaee>", line 3, in get_user_by_id
    assert isinstance(id, int), 'id должен быть целым числом'
AssertionError: id должен быть целым числом


`assert` нужен на этапе разработки, в реальном приложении его не желательно использовать.  
Он не предназначен для отлова его в блоке `try..except`.  
Также, он может быть отключен передачей интерпретатору опции `-O` и не будет сгенерирован:

  `$ python -O myapp.py`

## Производительность исключений

Исключения очень сильно влияют на производительность программы.  

**Пример 1.**

`try..except` обрабатываются медленно (дорого).  
Тест из 1 млн циклов:

In [18]:
import time

class Profiler():
    def __enter__(self):
        self._startTime = time.time()
    
    def __exit__(self, type, value, traceback):
        print ("Elapsed time: {:.3f} sec".format(time.time() - self._startTime))

my_dict = {'foo': 1}

with Profiler():
    for _ in range(1000000):
        try:
            my_dict['bar']
        except:
            pass

Elapsed time: 0.225 sec


**Пример 2.**

Здесь блок `try..except` заменен логическим выражением `if`, это в несколько раз быстрее.  
Тест из 1 млн циклов:

In [19]:
with Profiler():
    for _ in range(1000000):
        if 'bar' in my_dict:
            _ = my_dict['bar']

Elapsed time: 0.059 sec


## Итоги:

* Доступ к объекту исключения
* Генерация исключений, инструкция assert
* Исключения типа AssertionError
* Вопросы производительности
* Работа с пользовательскими исключениями (поговорим в следующем слайде)